# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate work to my team members in order to continuously monitor and analyze market data for AAPL.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL", "context": "The data analyst is responsible for using statistical modeling and machine learning to identify trends and predict market movements for AAPL. They need to provide insights and alerts about significant market opportunities or threats."}

> Entering new CrewAgentExecutor chain...
I should start by reading the latest market data for AAPL to identify any trends or patterns.

Action: Read website content
Action Input: {"website_url": "https://www.marketwatch

Final Answer: 
After analyzing the website content from Yahoo Finance for AAPL, the current stock price is $209.00 with a slight decrease of 0.02%. The stock has experienced a 1.05% decrease in the last trading session. With a market cap of 3.122T, Apple Inc. (AAPL) is a leading consumer electronics and technology company known for its iPhone, Mac, iPad, wearables, and services. The company's financial highlights include a profit margin of 24.30% and a return on equity of 138.01%. AAPL's trailing total returns as of 7/30/2025 are YTD 16.32%, 1-Year 4.01%, 3-Year 30.70%, and 5-Year 123.57%. This information can be used to provide insights and alerts for developing trading strategies for AAPL based on user-defined risk tolerance.

> Finished chain.
 

After analyzing the website content from Yahoo Finance for AAPL, the current stock price is $209.00 with a slight decrease of 0.02%. The stock has experienced a 1.05% decrease in the last trading session. With a market cap of 3.122T, Apple 

I need to gather more information on AAPL's historical performance, market trends, and any recent news that could impact its stock price in order to develop and refine trading strategies effectively.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL"} 

Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance Oops, something went wrong Skip to navigation Skip to main content Skip to right column News Today's news US Politics World Tech Audio Computing Gaming Wordle Home entertainment TVs Phones Science Streaming Streaming reviews VPN Wearables Deals Prime Day 2025 Best Amazon Prime Day deals More AI Apps AR and VR Business Cameras Cyber security Entertainment General Smart home Social media Transportation Weather Climate change Health Wellness Mental health Sexual health Dermatology Oral health Hair loss Foot health Nutrition Healthy eating Meal delivery Weight loss Vitamins and supplements Fitness Equipment Exercise Women’s h

Final Answer: After analyzing the website content from Yahoo Finance for AAPL, the current stock price is $209.00 with a slight decrease of 0.02%. The stock has experienced a 1.05% decrease in the last trading session. With a market cap of 3.122T, Apple Inc. (AAPL) is a leading consumer electronics and technology company known for its iPhone, Mac, iPad, wearables, and services. The company's financial highlights include a profit margin of 24.30% and a return on equity of 138.01%. AAPL's trailing total returns as of 7/30/2025 are YTD 16.32%, 1-Year 4.01%, 3-Year 30.70%, and 5-Year 123.57%. This information can be used to provide insights and alerts for developing trading strategies for AAPL based on user-defined risk tolerance.

> Finished chain.
 

After analyzing the website content from Yahoo Finance for AAPL, the current stock price is $209.00 with a slight decrease of 0.02%. The stock has experienced a 1.05% decrease in the last trading session. With a market cap of 3.122T, Apple I

Final Answer: 
A trading strategy is a systematic methodology used for buying and selling in the securities markets. It is based on predefined rules and criteria used when making trading decisions. Trading strategies can be simple or complex, and they consider factors such as investment style, market cap, technical indicators, fundamental analysis, risk tolerance, and more. The key is to set a trading strategy using objective data and analysis and adhere to it diligently. It should also be periodically re-evaluated and adjusted as needed. There are different types of trading strategies, including technical, fundamental, and quantitative strategies, each with its own approach to decision-making. Developing a trading strategy involves planning, placing trades, and managing them effectively to achieve desired outcomes. Trading strategies are used to avoid behavioral biases and ensure consistent results. It's important to stress-test trading strategies under varying market conditions to me

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for Proposed Trading Strategies and Execution Plans for AAPL:

Executive Summary:
The proposed trading strategies and execution plans for AAPL have been analyzed for potential risks and mitigation strategies. The analysis was conducted based on insights provided by the Data Analyst, user-defined risk tolerance (Medium), and trading preferences, such as Day Trading. The strategies aim to identify trends and predict market movements for AAPL.

Risk Analysis:
1. Market Risk:
- AAPL is subject to market volatility and macroeconomic factors that can impact its stock price.
- Mitigation Strategy: Diversify the portfolio to reduce exposure to AAPL-specific risks.

2. Liquidity Risk:
- Inadequate liquidity in the market may impact the ability to execute trades at desired prices.
- Mitigation Strategy: Use limit orders to manage liquidity risk effectively.

3. Operational Risk:
- Technical failures or disruptions in trading platforms may result in losses.
- Mitigation Strategy: Implement backup systems and conduct regular testing to ensure operational resilience.

4. Regulatory Risk:
- Changes in regulations or legal requirements may affect trading strategies for AAPL.
- Mitigation Strategy: Stay informed about regulatory developments and adapt strategies accordingly.

5. Credit Risk:
- Counterparty default risk in trading transactions may lead to financial losses.
- Mitigation Strategy: Perform credit checks on counterparties and set limits on exposure.

6. Systemic Risk:
- System-wide events, such as market crashes, can impact AAPL's stock price.
- Mitigation Strategy: Maintain a diversified portfolio to mitigate systemic risk.

Recommendations:
- Monitor market trends and news related to AAPL to stay informed.
- Regularly review and adjust trading strategies based on changing market conditions.
- Conduct stress tests to assess the robustness of the trading strategies under various scenarios.

Overall, the proposed trading strategies for AAPL should be carefully monitored and adjusted to mitigate risks effectively and achieve desired outcomes.